In [57]:
# importing libraries

import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import glob

# changing the file directory

os.chdir('E:\ml\Coding_problem\Coding_data')

In [58]:
# creating list for the file name of each customer data

xmlfiles = []
for file in glob.glob("*.xml"):
    xmlfiles.append(file)
    
xmlfiles

['customer1113697_loan1115483_crif_report.html.xml',
 'customer1129550_loan1131339_crif_report.html.xml',
 'customer1195586_loan1197471_crif_report.html.xml',
 'customer14235_loan14235_crif_report.html.xml',
 'customer16475_loan16475_crif_report.html.xml',
 'customer40409_loan40409_crif_report.html.xml',
 'customer773504_loan774538_crif_report.html.xml',
 'customer787561_loan788638_crif_report.html.xml',
 'customer794397_loan795497_crif_report.html.xml',
 'customer898231_loan899591_crif_report.html.xml']

In [59]:
#function to create raw dataframe

def xml_to_raw_df(file):

    # creating tree and root to parse the data from xml file 

    tree = ET.parse(file)
    root = tree.getroot()

    # parsing the useful information

    l1 = []                #empty list to contain the rows for all trades (to be converted to dataframe)

    for x in root.iter('LOAN-DETAILS'):
        l2 = []            #empty list to contain the each row

        ACCT_TYPE = x.find('ACCT-TYPE').text
        l2.append(ACCT_TYPE)
        
        if (x.find('DISBURSED-AMT') != None):      # checking for non-empty values
            DISBURSED_AMT = x.find('DISBURSED-AMT').text
            DISBURSED_AMT = float(DISBURSED_AMT.replace(',', ''))
        
        else:
            DISBURSED_AMT = float(0)              # if DISBURSED-AMT is not present in the XML child filling it as zero 


        l2.append(float(DISBURSED_AMT))              

        DISBURSED_DATE = x.find('DISBURSED-DATE').text
        l2.append(DISBURSED_DATE)

        
        if (x.find('COMBINED-PAYMENT-HISTORY').text != None):     # check for non-empty string COMBINED-PAYMENT-HISTORY
            COMBINED_PAYMENT_HISTORY = x.find('COMBINED-PAYMENT-HISTORY').text
        else:
            COMBINED_PAYMENT_HISTORY = 'xxxxxxxxxxxxxxxx'         # filling empty COMBINED-PAYMENT-HISTORY cell with dummy string
            
        l2.append(COMBINED_PAYMENT_HISTORY)
        l1.append(l2)



        #print( ACCT_TYPE, DISBURSED_AMT, DISBURSED_DATE,COMBINED_PAYMENT_HISTORY)
        #print('****************')

    # generating the dataframe from parsed data
    col = ['ACCT_TYPE', 'DISBURSED_AMT', 'DISBURSED_DATE','COMBINED_PAYMENT_HISTORY']

    df = pd.DataFrame(l1, columns = col)
    
    return df

In [60]:
df = xml_to_raw_df(xmlfiles[4]) # working for a particular file
df.head()

,ACCT_TYPE,DISBURSED_AMT,DISBURSED_DATE,COMBINED_PAYMENT_HISTORY
0,Used Car Loan,221850.0,22-05-2015,"Jan:2018,000/XXX|Dec:2017,000/XXX|Nov:2017,000..."
1,Gold Loan,54997.0,19-07-2017,"Apr:2018,000/XXX|Mar:2018,000/STD|Feb:2018,000..."
2,Credit Card,104929.0,28-12-2015,"Apr:2019,000/XXX|Mar:2019,000/XXX|Feb:2019,000..."
3,Personal Loan,10000.0,24-01-2019,"Jan:2019,000/XXX|"
4,Personal Loan,5000.0,26-08-2018,"Dec:2018,000/XXX|Nov:2018,000/XXX|Oct:2018,000..."


In [61]:
# generationg list by splitting the payment history for each month (separated by '|')

def payment_separator(df_raw):
    
    payment_text = []
    
    for i in range(len(df_raw)):
        
        txt = df_raw['COMBINED_PAYMENT_HISTORY'][i]

        if (txt):
            payment_text.append(txt.split('|'))
    
    
    # payment dataframe for each trade 
    combined_payment = pd.DataFrame(payment_text)
    
    # cleaning the dataframe
    combined_payment = combined_payment.replace(r'^\s*$', np.nan, regex=True) 
    combined_payment = combined_payment.fillna('xxxxxxxxxxxxxxxx')      # adding dummy string for empty cell error handling  
    
    return combined_payment

In [62]:
# monthwise payment dataframe

MW_payment = payment_separator(df)
MW_payment.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,"Jan:2018,000/XXX","Dec:2017,000/XXX","Nov:2017,000/XXX","Oct:2017,000/XXX","Sep:2017,000/XXX","Aug:2017,000/XXX","Jul:2017,000/XXX","Jun:2017,000/XXX","May:2017,000/XXX","Apr:2017,000/XXX",...,"Oct:2015,000/XXX","Sep:2015,000/XXX","Aug:2015,000/XXX","Jul:2015,000/XXX","Jun:2015,000/XXX",xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx
1,"Apr:2018,000/XXX","Mar:2018,000/STD","Feb:2018,000/STD","Jan:2018,000/STD","Dec:2017,000/STD","Nov:2017,000/STD","Oct:2017,000/STD","Sep:2017,000/STD","Aug:2017,000/STD","Jul:2017,000/STD",...,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx
2,"Apr:2019,000/XXX","Mar:2019,000/XXX","Feb:2019,000/XXX","Jan:2019,000/XXX","Dec:2018,000/XXX","Nov:2018,000/XXX","Oct:2018,000/XXX","Sep:2018,000/XXX","Aug:2018,000/XXX","Jul:2018,000/XXX",...,"Jan:2017,056/XXX","Dec:2016,057/XXX","Nov:2016,086/XXX","Oct:2016,056/XXX","Sep:2016,025/XXX","Aug:2016,000/XXX","Jul:2016,000/XXX","Jun:2016,000/XXX","May:2016,000/XXX",xxxxxxxxxxxxxxxx
3,"Jan:2019,000/XXX",xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,...,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx
4,"Dec:2018,000/XXX","Nov:2018,000/XXX","Oct:2018,000/XXX","Sep:2018,000/XXX","Aug:2018,000/XXX",xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,...,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxx


In [82]:
# iterating through each trade and fetching all required information


# function to check whether the DPD value is float or not  

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def cal(df):
    
    # empty lists to store DPD values and the values with DPD over 30 days 
    
    DPD = []
    DPD_over_30 = []
    
    for i in range(len(df)):
        
        DPD_val = 0
        DPD_li = []

        for j in range(len(df.iloc[i])):
            
            #print(df.iloc[i][j])

            if (isfloat(df.iloc[i][j][9:12]) and df.iloc[i][j][9:12] != 'xxxxxxxxxxxxxxxx'):
                if (int(df.iloc[i][j][9:12]) != 0):
                    
                    DPD_val = (int(df.iloc[i][j][9:12]))
                    DPD_li.append(DPD_val)
            
            else:

                DPD_li.append(0)
                
            #print(isfloat(df.iloc[i][j][9:12]))
            #print(df.iloc[i][j][9:12])
            #print('---------')
            
        DPD_30 = []
        
        for i in range (len(DPD_li)):

            DPD_30_above = 0

            if DPD_li[i] > 30:
                DPD_30_above += 1

            DPD_30.append(DPD_30_above)

        
        DPD.append(max(DPD_li))
        DPD_over_30.append(sum(DPD_30))
        
        #print('DPD value is :', DPD_val, 'and DPD list is :', DPD_li)
        #print('the maximum DPD is :', DPD)
        #print('the number of months exceeding 30+ DPD is :', sum(DPD_30))
        #print('*****************************************************')

    return (DPD, DPD_over_30)
            
            

In [68]:
DPD_cols = cal(MW_payment)
#DPD_cols

DPD value is : 0 and DPD list is : [0, 0, 0, 0, 0]
the maximum DPD is : [0]
the number of months exceeding 30+ DPD is : 0
*****************************************************
DPD value is : 0 and DPD list is : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
the maximum DPD is : [0, 0]
the number of months exceeding 30+ DPD is : 0
*****************************************************
DPD value is : 25 and DPD list is : [0, 0, 54, 56, 57, 86, 56, 25, 0]
the maximum DPD is : [0, 0, 86]
the number of months exceeding 30+ DPD is : 5
*****************************************************
DPD value is : 0 and DPD list is : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
the maximum DPD is : [0, 0, 86, 0]
the number of months exceeding 30+ DPD is : 0
*****************************************************
DPD value is : 0 and DPD list is : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [69]:
# function to add the columns in the raw dataframe

def col_adder(l2, df):
    df['DPD'] = l2[0]
    df['30+ DPD months'] = l2[1]
    
    return df

In [70]:
df = col_adder(DPD_cols, df) # adding the payment due and 30+ DPD months columns in the raw dataframe
df.head()

,ACCT_TYPE,DISBURSED_AMT,DISBURSED_DATE,COMBINED_PAYMENT_HISTORY,DPD,30+ DPD months
0,Used Car Loan,221850.0,22-05-2015,"Jan:2018,000/XXX|Dec:2017,000/XXX|Nov:2017,000...",0,0
1,Gold Loan,54997.0,19-07-2017,"Apr:2018,000/XXX|Mar:2018,000/STD|Feb:2018,000...",0,0
2,Credit Card,104929.0,28-12-2015,"Apr:2019,000/XXX|Mar:2019,000/XXX|Feb:2019,000...",86,5
3,Personal Loan,10000.0,24-01-2019,"Jan:2019,000/XXX|",0,0
4,Personal Loan,5000.0,26-08-2018,"Dec:2018,000/XXX|Nov:2018,000/XXX|Oct:2018,000...",0,0


In [71]:
# function to calculate the results (% of trades with 30+ DPD, sum of total disbursed amount, max number of months of 30+DPD)

def results(df):
    cust_li = []

    dpd_count = (df['DPD'] != 0).sum()
    percent_DPD = dpd_count/len(df) * 100     # % of trades with 30+ DPD

    total_disb_amt = df.DISBURSED_AMT.sum()   # sum of total disbursed amount
    
    max_DPD = df['30+ DPD months'].max()      # max number of months of 30+DPD

    cust_li.append(round(percent_DPD, 3))
    cust_li.append(total_disb_amt)
    cust_li.append(max_DPD)

    return cust_li

In [72]:
# results for single customer
result_li = results(df)
result_li

[15.385, 2868467.0, 5]

In [79]:
# generating dataframe for final results 

def result_df(result_li):
    #ar = np.reshape(result_li, (3, 1)).T   # if there is only one customer data, then uncomment this and replace result_li <-> ar 

    return (pd.DataFrame(result_li, columns = ['% of trades with 30+ DPD', 'Total disbursed amount for all loans',\
                                               'Max number of months of 30+ DPD']))

In [76]:
result1 = result_df(result_li)
result1

,% of trades with 30+ DPD,Total disbursed amount for all loans,Max number of months of 30+ DPD
0,15.385,2868467.0,5.0


In [83]:
# iterating through each customer file and storing results in a list

li = []

# this single loop will generate the final results for all the customer data (provided all the above functions are defined)

for i in xmlfiles:
    #print(i)
    
    df = xml_to_raw_df(i)                  # raw dataframe from xml file
    MW_payment = payment_separator(df)     # monthwise payment history 
    DPD_cols = cal(MW_payment)             # DPD and DPD with 30+ days list
    df = col_adder(DPD_cols, df)           # dataframe with DPD and DPD with 30+ days list
    result_li = results(df)                # list of results 
    #print(result_li)
    
    li.append(result_li)

print(li)



[[18.519, 1057320.0, 4], [46.154, 795601.0, 10], [70.588, 4471863.0, 3], [21.622, 8532117.0, 7], [15.385, 2868467.0, 5], [32.143, 1075308.0, 5], [24.324, 4133365.0, 36], [16.667, 691839.0, 2], [17.143, 910791.0, 6], [15.385, 12449012.0, 18]]


In [84]:
# list to dataframe for the combined results of all the customers

final_df = result_df(li)
final_df

,% of trades with 30+ DPD,Total disbursed amount for all loans,Max number of months of 30+ DPD
0,18.519,1057320.0,4
1,46.154,795601.0,10
2,70.588,4471863.0,3
3,21.622,8532117.0,7
4,15.385,2868467.0,5
5,32.143,1075308.0,5
6,24.324,4133365.0,36
7,16.667,691839.0,2
8,17.143,910791.0,6
9,15.385,12449012.0,18


In [86]:
# writing CSV file 

final_df.to_csv('result.csv', index = False)